In [11]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

# Initial data lookup
def get_shape(df):
    print('The shape of the dataframe is : '+ str(df.shape))
    
def get_head(df, rows = 5):
    print("########################################################################")
    print(df.head(rows))
    
# Missing Values
def get_missing_values_by_column(df):
    all_cols = df.columns.values
    print(all_cols)
    for col in all_cols:
        missing_percent = df[col].isnull().mean()
        if missing_percent > 0:
            print("Column " + col + " has " + str(float("{0:.2f}".format(missing_percent * 100))) + "% missing values" )
            
def get_float_val (df):
    df.replace(regex=True,inplace=True,to_replace='[^a-zA-Z0-9\n\.]',value='')
    df = df.astype(dtype='float')
    # since host left it blank..replacing null values with 0 
    return df.fillna(value = 0).copy()

def get_bool_val (df):
    df.replace(['t','f'],[1,0],inplace=True)
    df = df.astype(dtype='bool')
    # since host left it blank..replacing null values with 0 
    return df.fillna(value = 0).copy()


def count_days_since(oldDate, currentDate = datetime(2016,1,1,0,0,0)):
    return (currentDate-oldDate).days


def preprocess_features(df):
    '''
    From looking at each of the categorical variable we found that,
    1. experiences_offered, host_verifications, market, has_availability, requires_license columns has only one value hence we need to Drop these columns from dataframe.
    2. 'security_deposit', 'cleaning_fee', 'extra_people' these columns need to be converted to float value of $
    3. amenities need to converted to int where number is total number of amenities
    4. host_response_rate need to converted to float
    5. Transit need to converted to nminal var where NaN = 0 and everuthing else is 1
    6. host_since need to converted to diff between 01/01/2016 - host_since in days

    '''
    # Convert string to Float for 'security_deposit', 'cleaning_fee', 'extra_people',host_acceptance_rate
    df['host_acceptance_rate'] = get_float_val(df['host_acceptance_rate'])
    df['host_response_rate'] = get_float_val(df['host_response_rate'])
    df['security_deposit'] = get_float_val(df['security_deposit'])
    df['cleaning_fee'] = get_float_val(df['cleaning_fee'])
    df['extra_people'] = get_float_val(df['extra_people'])
    df['price'] = get_float_val(df['price'])  # there are no missing values here
    
    # Create Transit_available variable from transit and drop null rows
    df['Transit_available'] = ~ (df.transit.isnull())
    df = df.drop(['transit'],axis=1);

    # Create Facilities variable by counting number of facilities propvided
    df['Facilities'] = df.amenities.apply(lambda a : len(a))
    df = df.drop(['amenities'],axis=1);

    # Create Host_experience variable, experience in days, from host_since
    df['Host_experience'] = pd.to_datetime(df.host_since).apply(count_days_since)
    df = df.drop(['host_since'],axis=1);
    
    # Convert string t,f variables in boolean variables
    df['host_is_superhost'] = get_bool_val(df['host_is_superhost'])
    df['require_guest_profile_picture'] = get_bool_val(df['require_guest_profile_picture'])
    df['require_guest_phone_verification'] = get_bool_val(df['require_guest_phone_verification'])
    df['host_has_profile_pic'] = get_bool_val(df['host_has_profile_pic'])
    df['host_identity_verified'] = get_bool_val(df['host_identity_verified'])
    df['is_location_exact'] = get_bool_val(df['is_location_exact'])
    df['has_availability'] = get_bool_val (df['has_availability'])
    df['instant_bookable'] = get_bool_val(df['instant_bookable'])
    
    # Clean unnecessary feature to reduce dimntionality experiences_offered, host_verifications, market, has_availability, requires_license
    df = df.drop(['experiences_offered','host_verifications','market','has_availability','requires_license'],axis=1)

    return df


# Lets impute data
def impute_features(df):
    '''
    INPUT
    df - pandas dataframe 
    
    OUTPUT
    df - pandas dataframe with imputed data
    
    This function cleans df using the following steps to produce X and y:
    1. For each numeric variable in X, fill the column with the mean value of the column.
    2. Lets drop all the rows which has nan values.
    3. Create dummy variables for categorical variables
    '''
   
    # Fill numeric columns with the mean
    num_vars = df.select_dtypes(include=['float', 'int']).columns
    print(num_vars)
    for col in num_vars:
        df[col].fillna((df[col].mean()), inplace=True)
        
    # Dummy the categorical variables
    cat_vars = df.select_dtypes(include=['object']).copy().columns
    print(cat_vars)
    for var in  cat_vars:
        # for each cat add dummy var, drop original column
        df = pd.concat([df.drop(var, axis=1), pd.get_dummies(df[var], prefix=var, prefix_sep='_', drop_first=True)], axis=1)
    return df

# Lets Split data as per target and features
def split_data(df,target='price'):
    '''
    INPUT
    df - pandas dataframe 
    target - target var 
    
    OUTPUT
    X - A matrix holding all of the variables you want to consider when predicting the response
    y - the corresponding response vector
    
    This function cleans df using the following steps to produce X and y:
    1. Drop all the rows with no target
    2. Create X as all the columns that are not the target column
    3. Create y as the target column
    '''
    # Drop rows with missing salary values
    df = df.dropna(subset=[target], axis=0)
    y = df[target]
    
    #Drop respondent and expected salary columns
    df = df.drop([target], axis=1)
    
    X = df
    return X, y


def normalise_data(df):   
    # normalise data
    scaler = MinMaxScaler().fit(df)
    return scaler.transform(df)

In [8]:
def data_wrangling(df,selected_vars,target='price'):

    # remove irrelavant features from dataframe
    df = df[selected_vars].copy()

    # Preprocess the features as per required for analysis
    df = preprocess_features(df)

    # Null Values
    df.columns[df.isnull().mean() > 0]

    #Use the function impute all the Null, nan values
    df = impute_features(df) 

    #Split data
    X,y = split_data(df,target)
    
    return X,y

In [9]:
def plot_by_target(df,feature,target):
    '''
    This function plots target vs feature plot
    '''
    df[target] = get_float_val(df[target])
    df[target] = df[target].astype(dtype='float')
    df = df.dropna(subset=[target],axis=0)
    plotg = df.groupby([feature])[target].mean()
    import matplotlib.pyplot as plt
    _ = plotg.plot(kind='bar',figsize=(10,10),title='Mean '+target+' by ' +feature+ ' in Seattle 2016 airbnb',y= 'Mean '+target,legend=True)

In [10]:
def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df